In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import nltk
import string
import re
import time as t
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import lyricsgenius
genius = lyricsgenius.Genius("bWUqXEIz9jor2w6fi9sLD3KfilReDFPZL7Ox3OCkD_Sz8ICXaAFSTOUkH-tsv1oW")
genius.remove_section_headers = True
genius.skip_non_songs = True

In [ ]:
# The goal of this prototype is to fetch the lyrics of a specific song, in preparation for the full script that will iterate through the yearly data.

songdata = pd.read_csv('data/top10songs.csv')

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
cookies = { 'sessionid': '123..'}
i = 4 # Set i to the position in the songdata datafrae you would like to examine.
url = 'https://www.azlyrics.com/lyrics/'
artist = songdata['Artist'][i] # artist1 and artist2 are the most common ways azlyrics represents artist names. artist1 removes spaces while artist2 replaces                                  them with a dash.
artist1 = ''.join(artist.split()).lower().replace('\'','').replace('!','').replace('.','').replace('+','').replace('ñ','n').replace('é','e')
artist2 = artist.lower().replace('\'','').replace(' ','-').replace('!','').replace('.','').replace('+','').replace('ñ','n').replace('é','e')
artistl = []
if ', ' in artist: # Handles cases where there are three listed artists.
    artistl.append(artist1[:artist1.rfind(',')])
    artistl.append(artist1[artist1.find(',')+1:artist1.rfind('and')])
    artistl.append(artist1[artist1.rfind('and')+3:])
elif ' with ' in artist:
    artistl.append(artist1[:artist1.rfind('with')])
    artistl.append(artist2[:artist2.rfind('-with')])
elif ' and ' in artist: # Handles cases where there are two listed artists.
    artistl.append(artist1[:artist1.rfind('and')])
    artistl.append(artist2[:artist2.rfind('-and')])
    if ' featuring ' in artist: # Handles a case with multiple featured artists.
        artistl.append(artist1[:artist1.rfind('featuring')])
        artistl.append(artist2[:artist2.rfind('featuring')])
    artistl.append(artist1) # Cases where 'and' is in the artist name, ex. 'Tones and I'
elif 'bts' in artist1: # Handles that one k-pop group.
    artistl.append('bangtanboys')
elif ' featuring ' in artist: # Handles cases where there is a featured artist.
    artistl.append(artist1[:artist1.rfind('featuring')])
    artistl.append(artist2[:artist2.rfind('featuring')])
else: # Handles cases where there is one artist (which is most of them)
    artistl.append(artist1)
    artistl.append(artist2)
    artistl.append(artist1[artist1.find('the')+3:]) # Handles a rare case where Wikipedia includes a "the" in the artist name and azlyrics does not.
# The first three entries of the above list are standard single artist titles. The next three handle cases of collaborations between two artists. The next four after that handle collaborations between three artists. The last entry handles a rare case where Wikipedia includes a "the" in the artist name and azlyrics does not.
title = ''.join(songdata['Title'][i].split()).lower().replace('\'','').replace(',','').replace('!','').replace('ñ','n').replace('é','e') # azlyrics seems to represent all song titles in the same way.
titlel = [title, 'a' + title, 'the' + title] # However, there are very rare cases, such as 'Holly Jolly Christmas', where azlyrics includes an 'a' or a 'the' at the beginning of a song and Wikipedia does not. These cases will only be tested if all of the artist configurations fail.
if 'futsal' in title:
    titlel.insert(0,'futsalshuffle')
print(artistl)
print(titlel)

In [ ]:
geniusl = []
if ', ' in artist: # Handles cases where there are three listed artists.
    geniusl.append(artist[:artist.rfind(',')])
    geniusl.append(artist[artist.find(',')+1:artist1.rfind(' and ')])
    geniusl.append(artist[artist.rfind(' and ')+5:])
elif ' with ' in artist:
    geniusl.append(artist[:artist.rfind(' with')])
elif ' and ' in artist: # Handles cases where there are two listed artists.
    geniusl.append(artist[:artist.rfind(' and')])
    if ' featuring ' in artist: # Handles a case with multiple featured artists.
        geniusl.append(artist[:artist.rfind(' featuring ')])
    geniusl.append(artist) # Cases where 'and' is in the artist name, ex. 'Tones and I'
elif ' featuring ' in artist: # Handles cases where there is a featured artist.
    geniusl.append(artist[:artist.rfind(' featuring ')])
else: # Handles cases where there is one artist (which is most of them)
    if 'the' in artist:
        geniusl.append(artist[artist.find('the ')+4:])
    if 'The' in artist:
        geniusl.append(artist[artist.find('The ')+4:])
    geniusl.append(artist)
    if 'Show' in artist:
        geniusl.append(artist[artist.find('The ')+4:artist.rfind(' Show')]) # Handles some cases from the 50s and 60s.
geniusl

In [ ]:
success = False
for j in range(len(titlel)):
    for k in range(len(artistl)):
        if requests.get(url + artistl[k] + '/' + titlel[j] + '.html').status_code == 200:
            html = requests.get(url + artistl[k] + '/' + titlel[j] + '.html',cookies=cookies,headers=headers).text
            soup = BeautifulSoup(html, 'lxml')
            print(url + artistl[k] + '/' + titlel[j] + '.html')
            success = True
            t.sleep(3)
            break
        t.sleep(3)
    if success: # Escapes the outer loop when URL succeeds without using a second request.
        break
    if j == len(titlel) - 1:
        print('Song ' + songdata['Title'][i] + ' by ' + artist + ' at position ' + str(i) + ' not found!')

In [ ]:
success = False
for l in range(len(geniusl)):
    if genius.search_song(songdata['Title'][i].replace('\'',''),geniusl[l]) != None:
        currentsong = genius.search_song(songdata['Title'][i].replace('\'',''),geniusl[l])
        success = True
        t.sleep(3)
        break
    t.sleep(3)
if success == False:
    print('Song ' + songdata['Title'][i] + ' by ' + artist + ' at position ' + str(i) + ' not found!')

In [ ]:
artistl

In [ ]:
print(soup.prettify()[:10000])

In [ ]:
my_text = soup.find('div',attrs={'class': None}).text.replace('\n',' ').replace(',','')[3:] # Extract lyrics text from the azlyrics page and replace newlines with spaces. Also, eliminate the \r tag and space at the beginning.
token_text = word_tokenize(my_text)
token_text = [i.lower() for i in token_text if not re.fullmatch('[' + string.punctuation + ']+', i)] #Remove tokens that are just punctuation. This method preserves contractions like "n't".
token_text

In [ ]:
my_text = currentsong.lyrics
token_text = word_tokenize(my_text)
token_text = [i.lower() for i in token_text if not re.fullmatch('[' + string.punctuation + ']+', i)] #Remove tokens that are just punctuation. This method preserves contractions like "n't".

In [ ]:
port_stem = PorterStemmer() # The following code will stem the words in the lyrics. This means that, for example, "stop", "stopping", and "stops" will only count as one unique word.

stem_text = []

for i in range(len(token_text)):
    stem_text.append(port_stem.stem(my_text[i]))

print(stem_text)

In [ ]:
unique_count = len(set(stem_text))
unique_count

In [ ]:
len(my_text)